In [ ]:
!pip install trl peft accelerate bitsandbytes datasets accelerate

In [1]:
base_model_id = "microsoft/Phi-3-mini-4k-instruct"

In [8]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model,PeftModel
from trl import SFTTrainer

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [4]:
base_model.config.pretraining_tp = 1

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
sample_phi_chat = [{ "content": "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?", "role": "user" }, { "content": "This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.", "role": "assistant" }, { "content": "Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?", "role": "user" }, { "content": "Sure, here are the steps to enable the secondary image hover feature on your Collection pages and Featured Collections sections:\n\n1. Log in to your Shopify account and go to your Online Store.\n2. Click on Customize theme for the section-based theme you are using.\n3. Navigate to the Collection pages or Featured Collections section where you want to enable the secondary image hover feature.\n4. With the section open, click on the section's setting (gear) icon in the top-left corner.\n5. In the settings panel that appears, look for an option labeled 'Image display' or 'Image hover'.\n6. If available, select 'Show secondary image on hover'.\n7. Save the changes and preview the Collection/Featured Collection page to see the effect.\n\nIf you're having trouble finding the setting, the best thing to do is refer to your theme's documentation, since the location and labeling of settings can vary between themes.", "role": "assistant" }, { "content": "Can you provide me with a link to the documentation for my theme?", "role": "user" }, { "content": "I don't have access to your store's theme information. However, you can usually find the documentation for your theme by going to the shopify theme store, finding your theme and clicking on the 'support' link located in the bottom right corner of the page. Alternatively, you can do a google search for the name of your theme followed by 'documentation' or 'user guide'.", "role": "assistant" }, { "content": "Can you confirm if this feature also works for the Quick Shop section of my theme?", "role": "user" }, { "content": "The secondary image hover feature may or may not work for your Quick Shop section, depending on the configuration of your theme. Some themes include this feature in the Quick Shop section by default, while others may require additional customization. To check if this feature is available for the Quick Shop section of your theme, follow these steps:\n\n1. Go to the Quick Shop section where you would like to enable the feature. 2. Click on the Quick Shop settings icon (gear icon) and look for 'Image display' or 'Image hover'. 3. If available, select 'Show secondary image on hover'. 4. Save the changes. If this option is not available in your Quick Shop section settings, you may need to reach out to your theme developer for assistance with customizing your Quick Shop section to include this feature.", "role": "assistant" } ]

In [27]:
sample_phi_chat

[{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
  'role': 'user'},
 {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
  'role': 'assistant'},
 {'content': 'Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?',
  'role': 'user'},
 {'content': "Sure, here are

In [21]:
sample_phi_chat[0]

{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
 'role': 'user'}

In [22]:
sample_phi_chat[1]

{'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
 'role': 'assistant'}

In [26]:
df = pd.read_csv("final_data.csv")
data = Dataset.from_pandas(df)
data

Dataset({
    features: ['question', 'answer'],
    num_rows: 3972
})

In [36]:
def format_prompt(data):
    question = data['question']
    answer = data['answer']
    user_prompt = {"content":question,"role":'user'}
    assistant_prompt = {"content":answer,"role":'assistant'}
    return {"messages":[user_prompt,assistant_prompt]}

In [37]:
data = data.map(format_prompt)

Map:   0%|          | 0/3972 [00:00<?, ? examples/s]

In [38]:
data.to_pandas()['messages'].iloc[0]

array([{'content': 'Why are there still significant travel delays and cancellations after the snowstorm?', 'role': 'user'},
       {'content': "There are still significant travel delays and cancellations after the snowstorm because there have been over 12,000 flights cancelled since Friday, with 1,800 flights cancelled alone on the day of the report. Many travelers are struggling to get back home, and another nor'easter could hit as early as Friday. Airports and airlines are working to get back to normal, but it will take several days for all operations to return to normal.", 'role': 'assistant'}],
      dtype=object)

In [39]:
def apply_chat_template(
    data,
    tokenizer,
):
    messages = data["messages"]
    data["text"] = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=False)
    return data

In [40]:
dataset = data.map(
    apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer})

Map:   0%|          | 0/3972 [00:00<?, ? examples/s]

In [44]:
dataset.to_pandas()["text"]

0       <s><|user|>\nWhy are there still significant t...
1       <s><|user|>\nWhat challenges are people in New...
2       <s><|user|>\nHow are the two major candidates ...
3       <s><|user|>\nHow has Donald Trump's loyalty be...
4       <s><|user|>\nWhat two different methods are be...
                              ...                        
3967    <s><|user|>\nWhat is the main idea behind the ...
3968    <s><|user|>\nHow does Nancy Campbell respond t...
3969    <s><|user|>\nWhat is the scope of Nancy Campbe...
3970    <s><|user|>\nWhat is the main philosophy of Ca...
3971    <s><|user|>\nWhy is Campbell concerned about t...
Name: text, Length: 3972, dtype: object

In [48]:
config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"
    ],
    bias="none",
    lora_dropout=0,
    task_type="CAUSAL_LM",
)

model = get_peft_model(base_model, config)

In [49]:
training_config = {
    "bf16": True,
    "do_eval": False,
    "learning_rate": 1.0e-05,
    "log_level": "info",
    "logging_steps": 20,
    "logging_strategy": "steps",
    "lr_scheduler_type": "cosine",
    "num_train_epochs": 1,
    "max_steps": -1,
    "output_dir": "./checkpoint_dir",
    "overwrite_output_dir": True,
    "per_device_eval_batch_size": 4,
    "per_device_train_batch_size": 4,
    "remove_unused_columns": True,
    "save_steps": 100,
    "save_total_limit": 1,
    "seed": 0,
    "gradient_checkpointing": True,
    "gradient_checkpointing_kwargs":{"use_reentrant": False},
    "gradient_accumulation_steps": 1,
    "warmup_ratio": 0.2,
    "report_to":None
    }

In [50]:
train_conf = TrainingArguments(**training_config)

In [51]:
trainer = SFTTrainer(
    model=model,
    args=train_conf,
    train_dataset=dataset,
    max_seq_length=2048,
    dataset_text_field="text",
    tokenizer=tokenizer
)

Map:   0%|          | 0/3972 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Using auto half precision backend


In [52]:
train_result = trainer.train()

***** Running training *****
  Num examples = 3,972
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 993
  Number of trainable parameters = 17,825,792
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
You are not running the flash-attention implementation, expect numerical differences.


Step,Training Loss
20,2.798700
40,2.832800
60,2.730900
80,2.729700
100,2.508000
120,2.332700
140,2.326200
160,2.011100
180,1.947200
200,1.840700


Saving model checkpoint to ./checkpoint_dir/checkpoint-100
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/d269012bea6fbe38ce7752c8940fea010eea3383/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initia

In [53]:
metrics = train_result.metrics

In [ ]:
trainer.save_model("RedHenNewsLLM-3B")

In [ ]:
trainer.save_model()

In [57]:
import os
os.listdir("RedHenNewsLLM-3B")

['special_tokens_map.json',
 'tokenizer.model',
 'adapter_model.safetensors',
 'adapter_config.json',
 'README.md',
 'tokenizer_config.json',
 'training_args.bin',
 'tokenizer.json',
 'added_tokens.json']

In [58]:
from huggingface_hub import notebook_login

notebook_login()

## Push Adapters only

In [60]:
!huggingface-cli upload lucifertrj/redhen-lab-news-chat-3b-adapters ./RedHenNewsLLM-3B

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
adapter_model.safetensors:   0% 0.00/71.3M [00:00<?, ?B/s]
tokenizer.model:   0% 0.00/500k [00:00<?, ?B/s]

Upload 3 LFS files:   0% 0/3 [00:00<?, ?it/s]


training_args.bin: 100% 5.11k/5.11k [00:00<00:00, 16.3kB/s]
tokenizer.model: 100% 500k/500k [00:01<00:00, 283kB/s]
adapter_model.safetensors: 100% 71.3M/71.3M [00:11<00:00, 6.33MB/s]


Upload 3 LFS files: 100% 3/3 [00:11<00:00,  3.92s/it]
https://huggingface.co/lucifertrj/redhen-lab-news-chat-3b-adapters/tree/main/.


In [61]:
merge_model = PeftModel.from_pretrained(base_model, "RedHenNewsLLM-3B")
model = merge_model.merge_and_unload()

model_dir = "./RedHenNewsLLM-Chat"
model.save_pretrained(model_dir, safe_serialization=False)
tokenizer.save_pretrained(model_dir)

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Configuration saved in ./RedHenNewsLLM-Chat/config.json
Configuration saved in ./RedHenNewsLLM-Chat/generation_config.json
Model weights saved in ./RedHenNewsLLM-Chat/pytorch_model.bin
tokenizer config file saved in ./RedHenNewsLLM-Chat/tokenizer_config.json
Special tokens file saved in ./RedHenNewsLLM-Chat/special_tokens_map.json


('./RedHenNewsLLM-Chat/tokenizer_config.json',
 './RedHenNewsLLM-Chat/special_tokens_map.json',
 './RedHenNewsLLM-Chat/tokenizer.model',
 './RedHenNewsLLM-Chat/added_tokens.json',
 './RedHenNewsLLM-Chat/tokenizer.json')

In [62]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
 "./checkpoint_dir",
 low_cpu_mem_usage=True,
 return_dict=True,
 torch_dtype=torch.float16,
 device_map="auto",
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model",safe_serialization=True)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/d269012bea6fbe38ce7752c8940fea010eea3383/config.json
Model config Phi3Config {
  "_name_or_path": "microsoft/Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/Phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/Phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid_pdrop": 0.0,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Phi3ForCausalLM.

All the weights of Phi3ForCausalLM were initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Phi3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--Phi-3-mini-4k-instruct/snapshots/d269012bea6fbe38ce7752c8940fea010eea3383/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}

loading file tokenizer.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are resizing the embe

## Push the model

In [63]:
tokenizer.save_pretrained("merged_model")

tokenizer config file saved in merged_model/tokenizer_config.json
Special tokens file saved in merged_model/special_tokens_map.json


('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/tokenizer.model',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [64]:
!huggingface-cli upload lucifertrj/redhen-lab-news-chat-3b ./merged_model

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
model-00001-of-00002.safetensors:   0% 0.00/4.97G [00:00<?, ?B/s]
model-00002-of-00002.safetensors:   0% 0.00/2.67G [00:00<?, ?B/s]

Upload 3 LFS files:   0% 0/3 [00:00<?, ?it/s]


tokenizer.model:   0% 0.00/500k [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0% 2.77M/4.97G [00:00<15:30, 5.34MB/s]
model-00001-of-00002.safetensors:   0% 3.36M/4.97G [00:01<33:56, 2.44MB/s]
model-00001-of-00002.safetensors:   0% 5.37M/4.97G [00:01<17:21, 4.77MB/s]
tokenizer.model: 100% 500k/500k [00:01<00:00, 391kB/s]
model-00001-of-00002.safetensors:   0% 6.67M/4.97G [00:01<14:41, 5.63MB/s]
model-00001-of-00002.safetensors:   0% 7.96M/4.97G [00:01<12:18, 6.72MB/s]
model-00002-of-00002.safetensors:   0% 9.67M/2.67G [00:01<04:36, 9.63MB/s]
model-00001-of-00002.safetensors:   0% 13.9M/4.97G [00:01<05:18, 15.6MB/s]
model-00002-of-00002.

In [65]:
!nvidia-smi

Tue Jun  4 15:53:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   76C    P0              33W /  72W |  10635MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--